### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install mpi4py
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit


fatal: destination path 'baselines' already exists and is not an empty directory.
/project/baselines
Obtaining file:///project/baselines
  Preparing metadata (setup.py) ... done
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
  Using cached mpi4py-3.1.3.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for mpi4py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [148 lines of output]
      running bdist_wheel
      running build
      running build_src
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.7
      creating build/lib.linux-x86_64-3.7/mpi4py
      copying src/mpi4py/__main__.py -> build/lib.linux-x86_64-3.7/mpi4py


In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import DDPG
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

ImportError: cannot import name 'DDPG' from 'stable_baselines' (/opt/conda/lib/python3.7/site-packages/stable_baselines/__init__.py)

In [ ]:
%cd ..
logdir = "Training/Logs/DDPG/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
cicddos2019 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
cicdos2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
cicids2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ids2017/clean/cicids2017.feather")
data = pd.concat([cicddos2019], ignore_index=True)
print(len(data.columns))
data.head(5)


In [ ]:
cicdos2017_features = ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"]
cicids2017_features = ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"]
cicddos2019_features = ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"]
important_features = cicddos2019_features + ["Label", "Timestamp"] # Adding Timestamp for custom environment logic
important_features = list(set(important_features))
print(len(important_features))

In [ ]:
removable_features = data.columns ^ important_features
print(removable_features.shape)
skinny_data = data.drop(labels=removable_features, axis='columns')

In [ ]:
data.info()
data.describe()

Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [ ]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [ ]:
data.drop('Timestamp', inplace=True, axis=1)
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[skinny_data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

In [ ]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


In [ ]:
print(x_train.shape)

In [ ]:
print(x_train.index)
print(y_train.index)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


In [ ]:
print(x_train.index)
print(y_train.index)

In [ ]:
print(y_train)

In [ ]:
print(y_train.iloc[17162,:])

In [ ]:
print(y_train.index)
print(x_train.index)

### Custom environment

In [ ]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(67,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [ ]:
def ids_dqn():
#     logger.configure(dir=os.path.join(os.getcwd(),'Training/Logs/ids_dqn'), format_strs=['stdout', 'tensorboard'])
    env = IdsEnv(images_per_episode=4096)
#     env = bench.Monitor(env, logger.get_dir())

    model = DDPG(
        MlpPolicy, 
        env, 
        verbose=1,
        tensorboard_log=logdir,
        full_tensorboard_log=True,
    ).learn(
        total_timesteps=int(1.0e4),
        log_interval=1000,
    )

    model.save('ddpg_stableb2_ids.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = ids_dqn()
print("DQN Training Time:", time.time() - start_time)


In [ ]:
def ids_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model.predict(obs)[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

ids_dqn_eval(dqn_model)

In [ ]:
os.getcwd()

In [ ]:
!tensorboard --logdir {logdir}

##### 